In [1]:
import sys
sys.argv = ['']

In [4]:
import numpy as np
import pandas as pd
import os

In [3]:
import model_classes_
# import process_inputs_
import run_models_
import sage_conv_cat_

run_models_.py
Model date and time:
2024-10-29-at-13-41-56 


Cell line: E122
Task: Regression
ChIP-seq resolution: 10000


Training set: 70%
Validation set: 15%
Testing set: 15%


Model hyperparameters: 
Number of epochs: 1000
Learning rate: 0.0001
Number of graph convolutional layers: 2
Graph convolutional embedding size: 256
Number of linear layers: 3
Linear hidden layer size: 256

Model's state_dict:
conv1.lin_l.weight 	 torch.Size([256, 12])
conv1.lin_l.bias 	 torch.Size([256])
conv2.lin_l.weight 	 torch.Size([256, 512])
conv2.lin_l.bias 	 torch.Size([256])
lin1.weight 	 torch.Size([256, 256])
lin1.bias 	 torch.Size([256])
lin2.weight 	 torch.Size([256, 256])
lin2.bias 	 torch.Size([256])
lin3.weight 	 torch.Size([1, 256])
lin3.bias 	 torch.Size([1])


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000
Elapsed time: 

In [6]:
cell_lines = ['E116', 'E122', 'E123']

In [ ]:
def extract_parameters(cl, ):
    path = f'src/data/{cl}/saved_runs/'
    files = [f for f in os.listdir(directory) if f.endswith(file_extension)]

In [8]:
import torch

model_path = "data/E122/saved_runs/model_2024-10-28-at-19-43-36.pt"
model = torch.load(model_path)

print(model)

In [9]:
npz_path = "data/E122/saved_runs/model_2024-10-28-at-19-43-36_test_metrics.npz"
np.load(npz_path)

In [5]:
%run run_models_.py -c E122 -rf 0

run_models_.py
Model date and time:
2024-10-29-at-13-51-01 


Cell line: E122
Task: Classification
ChIP-seq resolution: 10000


Training set: 70%
Validation set: 15%
Testing set: 15%


Model hyperparameters: 
Number of epochs: 1000
Learning rate: 0.0001
Number of graph convolutional layers: 2
Graph convolutional embedding size: 256
Number of linear layers: 3
Linear hidden layer size: 256

Model's state_dict:
conv1.lin_l.weight 	 torch.Size([256, 12])
conv1.lin_l.bias 	 torch.Size([256])
conv2.lin_l.weight 	 torch.Size([256, 512])
conv2.lin_l.bias 	 torch.Size([256])
lin1.weight 	 torch.Size([256, 256])
lin1.bias 	 torch.Size([256])
lin2.weight 	 torch.Size([256, 256])
lin2.bias 	 torch.Size([256])
lin3.weight 	 torch.Size([2, 256])
lin3.bias 	 torch.Size([2])


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000
Elapsed ti

In [7]:
# run classifications and regressions on each cell line
for cl in cell_lines:
    for i in [0,1]:
        %run run_models_.py -c {cl} -rf {i}

run_models_.py
Model date and time:
2024-10-29-at-14-00-43 


Cell line: E116
Task: Classification
ChIP-seq resolution: 10000


Training set: 70%
Validation set: 15%
Testing set: 15%


Model hyperparameters: 
Number of epochs: 1000
Learning rate: 0.0001
Number of graph convolutional layers: 2
Graph convolutional embedding size: 256
Number of linear layers: 3
Linear hidden layer size: 256

Model's state_dict:
conv1.lin_l.weight 	 torch.Size([256, 12])
conv1.lin_l.bias 	 torch.Size([256])
conv2.lin_l.weight 	 torch.Size([256, 512])
conv2.lin_l.bias 	 torch.Size([256])
lin1.weight 	 torch.Size([256, 256])
lin1.bias 	 torch.Size([256])
lin2.weight 	 torch.Size([256, 256])
lin2.bias 	 torch.Size([256])
lin3.weight 	 torch.Size([2, 256])
lin3.bias 	 torch.Size([2])


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000
Elapsed ti

In [ ]:
cl_conversion_dct = {'E116': 'GM1287', 'E123': 'K562', 'E122': 'HUVEC'}